In [ ]:
!python --version

## 加载数据到内存

In [1]:
import librosa
import numpy as np
import os
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
data_set = []
label_set = []
label2id = {genre:i for i,genre in enumerate(genres)}
id2label = {i:genre for i,genre in enumerate(genres)}
print(label2id)

hop_length = 512
for g in genres:
    print(g)
    for filename in os.listdir(f'../dataset/genres/{g}/'):
        songname = f'../dataset/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        # y换成 s 即频谱图会更准确
        S, phase = librosa.magphase(librosa.stft(y))
        rmse = librosa.feature.rms(S=S)

        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)

        # 计算节奏信息
        # Compute local onset autocorrelation
        oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
        tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr,
                                       hop_length=hop_length)
        # Compute global onset autocorrelation
        ac_global = librosa.autocorrelate(oenv, max_size=tempogram.shape[0])
        ac_global = librosa.util.normalize(ac_global)

        to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)} {np.mean(ac_global)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        data_set.append([float(i) for i in to_append.split(" ")])
        label_set.append(label2id[g])

{'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
blues
classical
country
disco
hiphop
jazz
metal
pop
reggae
rock


### 手动打乱数据集

通过`get_state()`保存状态，`set_state()`重新载入状态，可以使得两个数组在保证对应关系不变的情况下，完成随机打乱。

In [2]:
state = np.random.get_state()
np.random.shuffle(data_set)
np.random.set_state(state)
np.random.shuffle(label_set)

## 创建数据集

In [30]:
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data_set, dtype = float))
y = np_utils.to_categorical(np.array(label_set))

In [31]:
print("X.shape: ", X.shape, " Y.shape:", y.shape)

X.shape:  (1000, 27)  Y.shape: (1000, 10)


### 将测试集和训练集分隔

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

## 创建模型

In [33]:
from keras import models
from keras.layers import Dense, Dropout
from keras import regularizers
def create_model():
    model = models.Sequential()
    model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.003)))
    model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.003)))
    model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.003)))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    return model
model = create_model()

这里创建了一个包含三个隐藏层的神经网络，最后一层输出的是分类层，因为是10类，所以最后一层是10个单元。（这里增加了一层Dropout减少数据过拟合）

## 编译模型

这里是一个分类问题，所以使用类别交叉熵函数`categorical_crossentropy`，然后优化器选择`Adam`，评价指标选择正确率。

In [34]:
# from keras import optimizers
# sgd = optimizers.SGD(lr=0.001, decay=1e-6)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## 训练与评估

接下来使用`fit`方法进行训练，训练50轮。

In [35]:
model.fit(X_train, y_train, epochs=50, batch_size=128)

Epoch 1/50
800/800 [==============================] - 0s 376us/step - loss: 3.1312 - accuracy: 0.1737
Epoch 2/50
800/800 [==============================] - 0s 63us/step - loss: 2.8554 - accuracy: 0.3137
Epoch 3/50
800/800 [==============================] - 0s 86us/step - loss: 2.6508 - accuracy: 0.3462
Epoch 4/50
800/800 [==============================] - 0s 84us/step - loss: 2.5348 - accuracy: 0.3738
Epoch 5/50
800/800 [==============================] - 0s 62us/step - loss: 2.4107 - accuracy: 0.4225
Epoch 6/50
800/800 [==============================] - 0s 80us/step - loss: 2.3164 - accuracy: 0.4850
Epoch 7/50
800/800 [==============================] - 0s 109us/step - loss: 2.2227 - accuracy: 0.5000
Epoch 8/50
800/800 [==============================] - 0s 74us/step - loss: 2.1420 - accuracy: 0.5175
Epoch 9/50
800/800 [==============================] - 0s 105us/step - loss: 2.0631 - accuracy: 0.5163
Epoch 10/50
800/800 [==============================] - 0s 99us/step - loss: 1.9631 - acc

使用`evaluate`方法进行评估。

In [36]:
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

200/200 [==============================] - 0s 325us/step
test_acc:  0.7200000286102295
